In [1]:
import requests

def get_wikidata_qid(language_name):
    wiki_api = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "titles": language_name,
        "prop": "pageprops",
        "ppprop": "wikibase_item",
        "format": "json"
    }
    res = requests.get(wiki_api, params=params).json()
    pages = res["query"]["pages"]
    qid = next(iter(pages.values()))["pageprops"]["wikibase_item"]
    return qid

qid = get_wikidata_qid("Spanish language")  # e.g. returns "Q1321"
print("Wikidata QID:", qid)


Wikidata QID: Q1321


In [2]:
wd_api = "https://www.wikidata.org/w/api.php"
params = {
    "action": "wbgetentities",
    "ids": qid,
    "props": "claims",
    "format": "json"
}
res = requests.get(wd_api, params=params).json()
claims = res["entities"][qid]["claims"]
print(f'claims {claims}')
# For genealogical info, look at claims for P279 (subclass of) and others.
family_claims = [c["mainsnak"]["datavalue"]["value"]["id"]
                 for c in claims.get("P279", [])]  # subclass-of family IDs
print("Parent families QIDs:", family_claims)


claims {'P2924': [{'mainsnak': {'snaktype': 'value', 'property': 'P2924', 'hash': '9304a63b8fdb32b0f3d7530716b360ee83740969', 'datavalue': {'value': '2024508', 'type': 'string'}, 'datatype': 'external-id'}, 'type': 'statement', 'qualifiers': {'P1810': [{'snaktype': 'value', 'property': 'P1810', 'hash': '27874e22c10fa901d9e14f943a626247c47391e6', 'datavalue': {'value': 'ИСПАНСКИЙ ЯЗЫК', 'type': 'string'}, 'datatype': 'string'}], 'P2241': [{'snaktype': 'value', 'property': 'P2241', 'hash': 'ab73ca934336656292af069f2278599cecf5770f', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 123562959, 'id': 'Q123562959'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}]}, 'qualifiers-order': ['P1810', 'P2241'], 'id': 'Q1321$2B4A35A6-BB74-4347-A41A-99F05C3DE22B', 'rank': 'deprecated'}], 'P373': [{'mainsnak': {'snaktype': 'value', 'property': 'P373', 'hash': '2e24dadd2f7ca75523269a796975598bb48ef7c2', 'datavalue': {'value': 'Spanish language', 'type': 'string'}, 'datatype': 's

In [3]:


sparql_url = "https://query.wikidata.org/sparql"
qid = "Q1321"  # Spanish
query = f"""
SELECT ?familyLabel WHERE {{
  wd:{qid} wdt:P279 ?family .           # get its parent family
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
"""
data = requests.get(sparql_url, params={'query': query, 'format': 'json'}).json()
for result in data['results']['bindings']:
    print("Family:", result['familyLabel']['value'])


Family: Castilian
Family: West Iberian


In [4]:
family_qid = "Q71924"  # Castilian languages
query = f"""
SELECT ?langLabel WHERE {{
  wd:{family_qid} wdt:P527 ?lang .       # has-part children
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
"""
data = requests.get(sparql_url, params={'query': query, 'format': 'json'}).json()
children = [res['langLabel']['value'] for res in data['results']['bindings']]
print("Languages in family:", children)


Languages in family: ['Spanish', 'Extremaduran', 'Judaeo-Spanish', 'Amazonic Spanish', 'Spanish-based creole languages', 'Barranquenho']
